In [21]:
import pandas as pd 

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [22]:
help(mean_squared_error)

Help on function mean_squared_error in module sklearn.metrics._regression:

mean_squared_error(y_true, y_pred, *, sample_weight=None, multioutput='uniform_average')
    Mean squared error regression loss.
    
    Read more in the :ref:`User Guide <mean_squared_error>`.
    
    Parameters
    ----------
    y_true : array-like of shape (n_samples,) or (n_samples, n_outputs)
        Ground truth (correct) target values.
    
    y_pred : array-like of shape (n_samples,) or (n_samples, n_outputs)
        Estimated target values.
    
    sample_weight : array-like of shape (n_samples,), default=None
        Sample weights.
    
    multioutput : {'raw_values', 'uniform_average'} or array-like of shape             (n_outputs,), default='uniform_average'
        Defines aggregating of multiple output values.
        Array-like value defines weights used to average errors.
    
        'raw_values' :
            Returns a full set of errors in case of multioutput input.
    
        'unifo

In [23]:
df = pd.read_parquet(r'C:\Users\Suchi_Kumari\mlflow\MLOps-1\Intro\source\yellow_tripdata_2023-01.parquet')

In [24]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [25]:
len(df.columns)

19

In [26]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df['duration'] = df.duration.dt.total_seconds() / 60

In [27]:
mean = df.duration.mean()
print(mean)

15.668995167330452


In [28]:
deviation = df.duration.std()
print(deviation)

42.59435124195458


In [29]:
len(df[(df.duration >= 1) & (df.duration <= 60)]) / len(df) * 100

98.1220282212598

In [30]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [31]:
categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)

In [47]:
train_dicts = df[categorical].to_dict(orient='records')

In [49]:
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
print(f'Feature matrix size: {X_train.shape}')

Feature matrix size: (3066766, 518)


In [35]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6133532 stored elements and shape (3066766, 518)>
  Coords	Values
  (0, 44)	1.0
  (0, 326)	1.0
  (1, 149)	1.0
  (1, 458)	1.0
  (2, 150)	1.0
  (2, 463)	1.0
  (3, 228)	1.0
  (3, 300)	1.0
  (4, 238)	1.0
  (4, 267)	1.0
  (5, 39)	1.0
  (5, 326)	1.0
  (6, 46)	1.0
  (6, 411)	1.0
  (7, 109)	1.0
  (7, 305)	1.0
  (8, 148)	1.0
  (8, 329)	1.0
  (9, 7)	1.0
  (9, 304)	1.0
  (10, 226)	1.0
  (10, 406)	1.0
  (11, 179)	1.0
  (11, 496)	1.0
  (12, 46)	1.0
  :	:
  (3066753, 329)	1.0
  (3066754, 156)	1.0
  (3066754, 496)	1.0
  (3066755, 204)	1.0
  (3066755, 307)	1.0
  (3066756, 51)	1.0
  (3066756, 326)	1.0
  (3066757, 243)	1.0
  (3066757, 403)	1.0
  (3066758, 55)	1.0
  (3066758, 327)	1.0
  (3066759, 65)	1.0
  (3066759, 484)	1.0
  (3066760, 47)	1.0
  (3066760, 403)	1.0
  (3066761, 204)	1.0
  (3066761, 267)	1.0
  (3066762, 234)	1.0
  (3066762, 272)	1.0
  (3066763, 151)	1.0
  (3066763, 274)	1.0
  (3066764, 238)	1.0
  (3066764, 402)	1.0
  (3066765, 4

In [36]:
target = 'duration'
y_train = df[target].values

In [42]:
import numpy as np
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mse = mean_squared_error(y_train, y_pred)
rmse = np.sqrt(mse)
print(f"Train RMSE: {rmse}")


Train RMSE: 41.99649771953289


In [43]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].astype('str')
    
    return df

In [45]:
df_val = read_data(r'C:\Users\Suchi_Kumari\mlflow\MLOps-1\Intro\source\yellow_tripdata_2023-02.parquet')
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val = df_val.duration.values
y_pred = lr.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
print(f"Train RMSE: {rmse}")

Train RMSE: 7.945205219955657
